another try with package - under construction

rank_bm25
maybe: 
bm25-pt
https://github.com/jxmorris12/bm25_pt

another try

In [ ]:
pip install rank_bm25

In [1]:
# Import necessary libraries
import json
import os
import re
import nltk
import math
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from collections import defaultdict
from rank_bm25 import BM25Okapi
from rank_bm25 import BM25L

#from tokenizers import Tokenizer

import bm25s


# Import nltk data
# https://www.nltk.org/data.html
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')  # Ensure stopwords are downloaded


# set data path

data_path_abstract = r"c:\Users\Adrian\Development\air\longeval_sci_training_2025_abstract\longeval_sci_training_2025_abstract\documents"

# for dev
#data_path_abstract = r"c:\Users\hubin\TULokal\AIRLocal\longeval_sci_training_2025_abstract\longeval_sci_training_2025_abstract\docShort"


data_path_abstract_q = r"C:\Users\Adrian\Development\air\longeval_sci_training_2025_abstract\longeval_sci_training_2025_abstract"

# os.path.join(data_path_abstract_q, file_name)
#data_folder = r"c:\Users\hubin\TULokal\AIRLocal\longeval_sci_training_2025_abstract\longeval_sci_training_2025_abstract\documents"


resource module not available on Windows


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adrian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Adrian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adrian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#pip install retriv
# Note: SearchEngine is an alias for the SparseRetriever
'''from retriv import SearchEngine

collection = [
  {"id": "doc_1", "text": "Generals gathered in their masses"},
  {"id": "doc_2", "text": "Just like witches at black masses"},
  {"id": "doc_3", "text": "Evil minds that plot destruction"},
  {"id": "doc_4", "text": "Sorcerer of death's construction"},
]

se = SearchEngine("new-index").index(collection)

se.search("witches masses")
'''

load data, tokenize 

In [2]:
# Load and preprocess data
# one stream 
def load_and_preprocess_data(filepath):
    def document_generator():
        stop_words = set(stopwords.words('english'))  # Load English stop words
        #stemmer = PorterStemmer()  # PorterStemmer 1/3 slower than SnowballStemmer
        stemmer = SnowballStemmer('english')  # Initialize nltk stemmer

        documents = []
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                doc = json.loads(line)
                text = f"{doc.get('title', '')} {doc.get('abstract', '')}"
                authors_text = ' '.join([author.get('name', '').lower() for author in doc.get('authors', [])])
                text = f"{text} {authors_text}" 
                text = text.lower()
                text = re.sub(r'\W+', ' ', text)
                tokens = word_tokenize(text)
                #filtered_tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
                # Filter tokens: remove stop words, single characters, and numbers
                filtered_tokens = [
                    token for token in tokens 
                    if token not in stop_words and len(token) > 1 and not token.isdigit()
                ]
                # Apply steming
                stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
                documents.append({
                    'id': doc.get('id'),
                    'tokens': stemmed_tokens
                })
    return document_generator()

def load_and_preprocess_data_no_tokenization(filepath):
    def document_generator():
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                doc = json.loads(line)
                text = f"{doc.get('title', '')} {doc.get('abstract', '')}"
                authors_text = ' '.join([author.get('name', '').lower() for author in doc.get('authors', [])])
                text = f"{text} {authors_text}"
                yield {
                    'id': doc.get('id'),
                    'text': text
                }
    return document_generator()

# returns documents with tokens per stream (=field)  
def load_and_preprocess_data_BM25F(filepath):
    stop_words = set(stopwords.words('english'))  # Load English stop words
    #stemmer = PorterStemmer()  # PorterStemmer 1/3 slower than SnowballStemmer
    stemmer = SnowballStemmer('english')  # Initialize nltk stemmer

    documents = []
    #count_per_field = {'title': 0, 'abstract': 0, 'authors': 0}  # Initialize counts

    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            doc = json.loads(line)
            
            # title tokenization
            titel_text = doc.get('title', '').lower()       # convert to lowercase
            titel_text = re.sub(r'\W+', ' ', titel_text)    # replace non-word characters with space
            title_tokens = word_tokenize(titel_text)
            # Filter tokens: remove stop words, single characters, and numbers
            title_tokens = [
                token for token in title_tokens
                if token not in stop_words and len(token) > 1 and not token.isdigit()
            ]
            title_tokens = [stemmer.stem(token) for token in title_tokens]

            # abstract tokenization
            abstract_text = doc.get('abstract', '') or ''  # convert missing to empty string
            abstract_text = abstract_text.lower()
            abstract_text = re.sub(r'\W+', ' ', abstract_text)
            abstract_tokens = word_tokenize(abstract_text)
            # Filter tokens: remove stop words, single characters, and numbers
            abstract_tokens = [
                token for token in abstract_tokens
                if token not in stop_words and len(token) > 1 and not token.isdigit()
            ]
            abstract_tokens = [stemmer.stem(token) for token in abstract_tokens]

            # authors tokenization
            # potential: optimize for names of persons
            authors_text = ' '.join([author.get('name', '').lower() for author in doc.get('authors', [])])
            authors_text = re.sub(r'\W+', ' ', authors_text)
            authors_tokens = word_tokenize(authors_text)
            # Filter tokens: remove stop words, single characters, and numbers
            authors_tokens = [
                token for token in authors_tokens
                if token not in stop_words and len(token) > 1 and not token.isdigit()
            ]
            authors_tokens = [stemmer.stem(token) for token in authors_tokens]
            
            # Combine all tokens
            documents.append({
                'id': doc.get('id'),
                #'tokens': stemmed_tokens,
                'title': title_tokens,
                'abstract': abstract_tokens,
                'authors': authors_tokens
            })

            # calculate number of tokens per field
            #count_per_field['title'] += len(title_tokens)
            #count_per_field['abstract'] += len(abstract_tokens)
            #count_per_field['authors'] += len(authors_tokens)
            
    # number of documents
    #doc_count = len(documents)
    return documents #, doc_count, count_per_field

In [3]:
# Function to process all files in a folder
def load_and_preprocess_folder(folder_path):
    def folder_generator():
        for filename in os.listdir(folder_path):
            if filename.endswith('.jsonl'):
                filepath = os.path.join(folder_path, filename)
                print(f"Processing file: {filepath}")
                yield from load_and_preprocess_data(filepath)
    return folder_generator()

In [4]:
#data_folder = r"c\Users\Adrian\Development\air\longeval_sci_training_2025_abstract\longeval_sci_training_2025_abstract\documents"
all_documents = load_and_preprocess_folder(data_path_abstract)
#print(f"Loaded {len(all_documents)} documents from folder.")

In [5]:
from rank_bm25 import BM25Okapi
from rank_bm25 import BM25L
# from nltk.tokenize.destructive import NLTKWordTokenizer
import numpy as np

tokenized_docs = [doc['tokens'] for doc in all_documents]

bm25L = BM25L(tokenized_docs) # takes lenght of documents into account
#bm25 = BM25Okapi(tokenized_docs)


Processing file: c:\Users\Adrian\Development\air\longeval_sci_training_2025_abstract\longeval_sci_training_2025_abstract\documents\documents_000001.jsonl


TypeError: 'NoneType' object is not iterable

In [ ]:

#query = "retrieval quantum"
#tokenized_query = query.split(" ")
tokenized_query = ["retrieval", "quantum"]


scoresBM25L = bm25L.get_scores(tokenized_query)
#scoresBM25 = bm25.get_scores(tokenized_query)
#print(scoresBM25L)

ranked = bm25L.get_top_n(tokenized_query, all_documents, n=2)
#r#anked_basic = bm25.get_top_n(tokenized_query, all_documents, n=2)

print(ranked)
#print(ranked_basic)

ranked_doc_ids = [doc['id'] for doc in ranked]
#ranked_doc_ids_basic = [doc['id'] for doc in ranked_basic]


print(ranked_doc_ids)
#print(ranked_doc_ids_basic)





# Evaluate 

- Search Information includes i) unique (anonymous) identifiers for individual user session; ii) search query; iii) returned results.
- Click Information records, for each click, i) a unique (anonymous) identifier for individual user session; ii) the link that was clicked in the results list; iii) the position of clicked link in results list.

queries:
training queries
│-- queries.txt # Tab-separated plain text file with queries and IDs 
- ID, search query

qrels:
│-- qrels.txt # Relevance judgments file in TREC format 
click information 
- ID, datum, dokumentID, relevanz

(1) nDCG scores calculated on provided test sets. Such a classical evaluation measure is consistent with Web search, for which the discount emphasises the ordering of the top results.

(2) Relative nDCG Drop (RnD) measured by computing the difference between snapshots test sets. This measure supports the evaluation of the impact of the data changes on the systems’ results.

In [ ]:
# Parse Queries.txt
def load_queries(filepath):
    queries = {}
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split()
            query_id = parts[0]
            query_text = parts[1]
            queries[query_id] = query_text.split()  # Tokenize query
    return queries



# Parse qrels.txt
def load_qrels(filepath):
    qrels = defaultdict(dict)
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split()
            query_id = parts[0]
            doc_id = parts[2]
            relevance = int(parts[3])
            qrels[query_id][doc_id] = relevance
    return qrels

# Load files
data_path_queries = os.path.join(data_path_abstract_q, "queries.txt")
data_path_qrels = os.path.join(data_path_abstract_q, "qrels.txt")

queries = load_queries(data_path_queries)
qrels = load_qrels(data_path_qrels)